In [193]:
import altair as alt
import httpx
import numpy as np
import polars as pl
import pycountry
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from utils import (
        compress,
        decompress,
        fetch_acled_for_countries,
        numeric_iso_col,
        year_month_col,
)


In [194]:
pycountry.countries.get(alpha_3='ABW')

Country(alpha_2='AW', alpha_3='ABW', flag='🇦🇼', name='Aruba', numeric='533')

In [195]:
kei = pl.read_csv('data/oecd_big_kei.csv')

In [196]:
num_kei = numeric_iso_col(kei, 'REF_AREA').drop_nulls('iso')

In [197]:
num_kei_keep = num_kei['iso', 'TIME_PERIOD', 'Measure', 'OBS_VALUE'].with_columns(
    pl.concat_str([pl.col('TIME_PERIOD'),
                   pl.col('iso')],
                   separator='-')
                   .alias('date_iso')
                   )

In [198]:
num_kei_keep

iso,TIME_PERIOD,Measure,OBS_VALUE,date_iso
str,str,str,f64,str
"""578""","""2024-04""","""Production volume""",2.764067,"""2024-04-578"""
"""578""","""2024-03""","""Production volume""",4.253215,"""2024-03-578"""
"""578""","""2020-09""","""Retail trade volume""",9.257266,"""2020-09-578"""
"""578""","""2020-08""","""Retail trade volume""",8.905579,"""2020-08-578"""
"""578""","""2020-07""","""Retail trade volume""",15.236052,"""2020-07-578"""
…,…,…,…,…
"""410""","""2019-07""","""Unemployment""",4.0,"""2019-07-410"""
"""410""","""2019-08""","""Unemployment""",3.6,"""2019-08-410"""
"""410""","""2019-09""","""Unemployment""",3.6,"""2019-09-410"""


In [199]:
num_kei_unique = num_kei_keep.unique()

In [200]:
num_kei_unique

iso,TIME_PERIOD,Measure,OBS_VALUE,date_iso
str,str,str,f64,str
"""124""","""2024-08""","""Production volume""",-2.721225,"""2024-08-124"""
"""124""","""2021-03""","""Permits issued for dwellings""",180.576531,"""2021-03-124"""
"""233""","""2023-01""","""Immediate interest rates, call…",1.901312,"""2023-01-233"""
"""203""","""2018-08""","""Composite consumer confidence""",0.8,"""2018-08-203"""
"""826""","""2022-09""","""Production volume""",-3.784861,"""2022-09-826"""
…,…,…,…,…
"""040""","""2023-11""","""Production volume""",124.536164,"""2023-11-040"""
"""428""","""2020-12""","""Retail trade volume""",-3.938521,"""2020-12-428"""
"""724""","""2022-05""","""Unemployment""",12.9,"""2022-05-724"""


In [201]:
num_kei_pivot = num_kei_unique.pivot(
    index=['iso', 'TIME_PERIOD'],
    on='Measure',
    values='OBS_VALUE',
    aggregate_function='first')

In [202]:
num_kei_pivot.shape

(3948, 24)

In [203]:
#new_acled = fetch_acled_for_countries(num_kei_pivot)

In [204]:
new_acled_iso_str = new_acled.with_columns(
    pl.col('iso').cast(pl.String).str.zfill(3))

In [205]:
new_acled_iso_str.write_csv('data/inspected/acled_ocled_new.csv')

In [206]:
#decompress('data/inspected/acled_oecd_countries_2018-2024.csv.xz')
#acled = pl.read_csv('data/inspected/acled_oecd_countries_2018-2024.csv', schema_overrides={'iso': pl.String})

In [207]:
acled_year_month = new_acled_iso_str.with_columns(
    pl.col('event_date')
    .str.to_date()
    .dt.strftime("%Y-%m")
    .alias('year_month'))
acled_grouped = acled_year_month.select(['year_month',
                                         'iso'])
acled_data = acled_grouped.group_by(['year_month',
                               'iso']).len().rename({'len':'incidents'})

In [ ]:
acled_incidents = acled_data.select(
    ['iso', 'year_month', 'incidents']).unique().sort(['iso', 'year_month'])

In [ ]:
num_kei_year_month = num_kei_pivot.rename({'TIME_PERIOD': 'year_month'})
data = acled_incidents.join(
    num_kei_year_month, on=['year_month', 'iso'], how='left')

In [210]:
data

iso,year_month,incidents,Production volume,Permits issued for dwellings,"Immediate interest rates, call money, interbank rate",Composite consumer confidence,Retail trade volume,Merchandise imports,Share prices,Passenger car registrations,Consumer prices,M3,Merchandise exports,Nominal exchange rates,Short-term interest rates,Producer prices,Composite business confidence,Long-term interest rates,Unemployment,M1,Hourly earnings,Employment,Reference series (GDP),Composite leading indicator (CLI)
str,str,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""032""","""2018-01""",115,null,null,null,null,null,6.158642,null,null,25.006226,null,9.725997,19.018799,null,null,null,null,null,null,null,null,null,null
"""032""","""2018-02""",131,null,null,null,null,null,-2.40281,null,null,25.437113,null,11.488463,19.839243,null,null,null,null,null,null,null,null,null,null
"""032""","""2018-03""",231,null,null,null,null,null,6.178861,null,null,2.341063,null,13.607041,20.230331,null,null,null,null,null,null,null,null,null,null
"""032""","""2018-04""",247,null,null,null,null,null,17.023721,null,null,25.498157,null,-4.389221,20.241778,null,null,null,null,null,null,null,null,null,null
"""032""","""2018-05""",193,null,null,null,null,null,5.937603,null,null,26.290757,null,-1.047028,23.615701,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""840""","""2024-08""",809,-0.691184,null,5.33,73.056577,-1.167441,272.777,16.450962,-5.724336,2.53073,175.722216,176.715,null,5.12,null,-5.0,3.87,4.2,598.649901,140.156623,161425.0,100.421483,99.708014
"""840""","""2024-09""",873,-0.619829,null,4.83,75.423653,0.648368,3.01785,3.080177,-5.968293,2.440633,2.665058,-1.570891,null,4.86,null,-5.0,3.72,4.1,0.362607,140.759663,161802.0,100.388288,99.826562
"""840""","""2024-10""",1052,-1.467056,null,4.83,75.85403,0.284835,268.699,2.038868,40.459587,2.597905,3.133323,-2.690024,null,4.62,null,-6.2,4.1,4.1,602.885506,5.367868,161456.0,100.317277,99.970774


In [211]:
X = data.drop(['iso', 'year_month', 'incidents'])
y = data['incidents']
X_train, X_test, y_train,  y_test = train_test_split(X, y)

In [212]:
pipe = Pipeline([('imputer', SimpleImputer(strategy='mean')),
                 #('scaler', StandardScaler()),
                 ('model', RandomForestRegressor())])

In [213]:
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.8027629201946549

In [216]:
data.write_csv('data/cleaned/data.csv')
compress('data/cleaned/data.csv')

PosixPath('data/cleaned/data.csv.xz')

### Global Acute Food Insecurity
* https://data.humdata.org/dataset/global-acute-food-insecurity-country-data
* Only covers developing nations, not compatible with current OECD countries

In [ ]:
#ipc = pl.read_csv('data/ipc_global_national_long_latest_food_insecurity.csv')

### Global Mobility Report (Google):
* https://data.humdata.org/dataset/google-mobility-report
* Only covers 2020-2022

In [218]:
#global_mobility_report = pl.read_csv('data/inspected/Global_Mobility_Report.csv')

In [219]:
nightsky = pl.read_csv('data/ntl_adm2_monthly_2016.csv', null_values='NA')

In [220]:
def feature_importances(pipe) -> pl.DataFrame | None:
        model = pipe.named_steps['model']
        cols = pipe.named_steps['imputer'].get_feature_names_out()
        
        if hasattr(model, "coef_"):
            values = np.transpose(model.coef_)
            name = "coefficient"
        elif hasattr(model, "feature_importances_"):
            values = model.feature_importances_.reshape(-1, 1)
            name = "importance"
        else:
            return None

        return pl.DataFrame({ "feature": cols, name: values.flatten() })

In [229]:
importances = feature_importances(pipe).sort(by='importance')

In [230]:
importances.plot.bar(x='feature', y='importance')

alt.Chart(...)

In [7]:
nightsky

ISO_A3,ADM1CD_c,ADM2CD_c,NAM_0,NAM_1,NAM_2,date,n_gasflaring_locs,ntl_quality_prop_na,ntl_quality_prop_0_good,ntl_quality_prop_1_poor,ntl_quality_prop_2_gapfilled,ntl_sum,ntl_mean,ntl_median,ntl_max,ntl_q05,ntl_q95,ntl_gf_5km_sum,ntl_gf_5km_mean,ntl_gf_5km_median,ntl_gf_5km_max,ntl_gf_5km_q05,ntl_gf_5km_q95,ntl_nogf_5km_sum,ntl_nogf_5km_mean,ntl_nogf_5km_median,ntl_nogf_5km_max,ntl_nogf_5km_q05,ntl_nogf_5km_q95,ntl_gf_10km_sum,ntl_gf_10km_mean,ntl_gf_10km_median,ntl_gf_10km_max,ntl_gf_10km_q05,ntl_gf_10km_q95,ntl_nogf_10km_sum,ntl_nogf_10km_mean,ntl_nogf_10km_median,ntl_nogf_10km_max,ntl_nogf_10km_q05,ntl_nogf_10km_q95
str,str,str,str,str,str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-01-01""",0,0.0,0.874351,0.125649,0.0,5212.629395,6.044495,2.521918,90.729057,0.070298,22.623181,null,null,null,null,null,null,5212.629395,6.044495,2.521918,90.729057,0.070298,22.623181,null,null,null,null,null,null,5212.629395,6.044495,2.521918,90.729057,0.070298,22.623181
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-02-01""",0,0.0,0.906542,0.09242,0.001038,5044.547852,5.84959,2.771983,84.466125,0.096552,20.592231,null,null,null,null,null,null,5044.547852,5.84959,2.771983,84.466125,0.096552,20.592231,null,null,null,null,null,null,5044.547852,5.84959,2.771983,84.466125,0.096552,20.592231
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-03-01""",0,0.0,0.877466,0.122534,0.0,5555.727051,6.442346,3.145681,124.96019,0.181422,21.818279,null,null,null,null,null,null,5555.727051,6.442346,3.145681,124.96019,0.181422,21.818279,null,null,null,null,null,null,5555.727051,6.442346,3.145681,124.96019,0.181422,21.818279
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-04-01""",0,0.0,0.0,1.0,0.0,5187.452148,6.0153,2.89655,69.404099,0.24911,20.452081,null,null,null,null,null,null,5187.452148,6.0153,2.89655,69.404099,0.24911,20.452081,null,null,null,null,null,null,5187.452148,6.0153,2.89655,69.404099,0.24911,20.452081
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-05-01""",0,0.0,0.0,0.876428,0.123572,5010.268555,5.80984,3.408621,94.178596,0.210237,19.1727,null,null,null,null,null,null,5010.268555,5.80984,3.408621,94.178596,0.210237,19.1727,null,null,null,null,null,null,5010.268555,5.80984,3.408621,94.178596,0.210237,19.1727
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZWE""","""ZWE008""","""ZWE008003""","""Zimbabwe""","""Matabeleland North""","""Hwange""","""2016-12-01""",0,0.000047,0.0,0.683857,0.316095,1830.144897,0.012482,0.251133,105.533417,0.025113,0.477152,null,null,null,null,null,null,1830.144897,0.012482,0.251133,105.533417,0.025113,0.477152,null,null,null,null,null,null,1830.144897,0.012482,0.251133,105.533417,0.025113,0.477152
"""ZWE""","""ZWE009""","""ZWE009005""","""Zimbabwe""","""Matabeleland South""","""Matobo""","""2016-12-01""",0,0.0,0.000108,0.96191,0.037982,153.748627,0.004247,0.252224,28.51372,0.025222,0.479225,null,null,null,null,null,null,153.748627,0.004247,0.252224,28.51372,0.025222,0.479225,null,null,null,null,null,null,153.748627,0.004247,0.252224,28.51372,0.025222,0.479225
"""ZWE""","""ZWE009""","""ZWE009007""","""Zimbabwe""","""Matabeleland South""","""Bulilima""","""2016-12-01""",0,0.0,0.000526,0.987076,0.012397,85.868095,0.002562,0.256001,3.067989,0.0256,0.486401,null,null,null,null,null,null,85.868095,0.002562,0.256001,3.067989,0.0256,0.486401,null,null,null,null,null,null,85.868095,0.002562,0.256001,

In [4]:
nightsky_date = year_month_col(nightsky, 'date')

In [6]:
nightsky_date['ISO_A3', 'ntl_sum', 'year_month'].group_by('ISO_A3', 'year_month').mean()

ISO_A3,year_month,ntl_sum
str,str,f64
"""BRB""","""2016-05""",927.857848
"""BRN""","""2016-09""",1757.215505
"""BTN""","""2016-10""",37.516818
"""BVT""","""2016-07""",1.92469
"""CCK""","""2016-03""",0.0
…,…,…
"""ZAF""","""2016-01""",29690.783609
"""ZAF""","""2016-05""",30393.661642
"""ZAF""","""2016-12""",30225.746505
